<a href="https://colab.research.google.com/github/TonyStone23/UAV-for-Concrete-Crack-Inspection/blob/main/code/Concrete_inspection_UAV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Concrete Inspection Using UAV
* CSC-499
* Tony Stone & Dr. Alaa Sheta


In [15]:
# Install ultralytics
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.7 MB/s eta 0:00:00


In [20]:
# Importing Necessary Libraries
import pandas as pd
import numpy as np
import random as r
import matplotlib.pyplot as plt
import cv2 as cv
import os
import shutil
from ultralytics import YOLO
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
r.seed(42)
np.random.seed(42)

In [23]:
# Collect images from origional Mendeley Dataset, and create a Subset
source_directory = "/content/drive/MyDrive/Mendeley Data"
destination_directory = "/content/Mendeley Subset"

# The class folder names in your dataset (check them!)
classes = ["Negative", "Positive"]

# number of images per class to keep
n_samples = 200

# Clear destination if it already exists
if os.path.exists(destination_directory):
    shutil.rmtree(destination_directory)

for cls in classes:
    src_class_dir = os.path.join(source_directory, cls)
    images = [f for f in os.listdir(src_class_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    r.shuffle(images)

    # Limit to n_samples if class has more images
    images = images[:n_samples]

    # Split into 80% train, 20% val
    split_idx = int(0.8 * len(images))
    train_imgs = images[:split_idx]
    val_imgs = images[split_idx:]

    # Make destination folders
    train_class_dir = os.path.join(destination_directory, "train", cls)
    val_class_dir = os.path.join(destination_directory, "val", cls)
    os.makedirs(train_class_dir, exist_ok=True)
    os.makedirs(val_class_dir, exist_ok=True)

    # Copy files
    for img in train_imgs:
        shutil.copy(os.path.join(src_class_dir, img), os.path.join(train_class_dir, img))

    for img in val_imgs:
        shutil.copy(os.path.join(src_class_dir, img), os.path.join(val_class_dir, img))

print("Subset created at:", destination_directory)
# Note: Running this for n = 200 took roughly 5 minutes

Subset created at: /content/Mendeley Subset


In [27]:
# Train the YOLO Model on the subset
!yolo task=classify mode=train model=yolov8n-cls.pt data='Mendeley Subset' \imgsz=227 epochs=1 batch=64

Ultralytics 8.3.192 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=Mendeley Subset, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=227, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profil

In [35]:
# Load the trained model
model = YOLO('runs/classify/train3/weights/best.pt') # Make sure it loads from the right place ("train_" can be different after each run)

In [36]:
# Connect to the drone video Stream
stream = " ACTUAL STREAM URL GOES HERE "
cap = cv.VideoCapture(stream)

In [37]:
# Main loop
while True:
    ret, frame = cap.read()
    if not ret:
        print("ret = False")
        break

    # Convert 'frame' into RGB format to be suitable for YOLO

    # Make prediction
        pred = model.predict(frame, imgsz = 224, verbose = False)[0]
    # Evaluate prediction
        class_id = int(pred.probs.top1)
        classification = model.names[class_id]
        confidence = float(pred.probs.top1conf)
    # Display

    if cv.waitKey(1) & 0xFF == ord('q'):
        break

# Clean-up
cap.release()
cv.destroyAllWindows()

ret = False
